# XGBoost, LightGBM and Catboost

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# pip install xgboost
# pip install lightgbm
# pip install catboost

In [3]:
# 데이터 불러오기
data = pd.read_csv("./data/otto_train.csv")
data.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,4,1,0,0,1,6,1,5,0,0,...,0,1,2,0,0,0,0,0,0,Class_1
4,5,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


In [4]:
data = data.drop(['id'], axis = 1) # id 제거

In [5]:
nCar = data.shape[0]
nVar = data.shape[1]
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

nCar: 61878 nVar: 94


## 타겟 변수의 문자열을 숫자로 변환

In [6]:
mapping_dict = {"Class_1": 1,
                "Class_2": 2,
                "Class_3": 3,
                "Class_4": 4,
                "Class_5": 5,
                "Class_6": 6,
                "Class_7": 7,
                "Class_8": 8,
                "Class_9": 9}
after_mapping_target = data['target'].apply(lambda x: mapping_dict[x])

In [7]:
feature_columns = list(data.columns.difference(['target']))

X = data[feature_columns]
y = after_mapping_target

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(49502, 93) (12376, 93) (49502,) (12376,)


## 1. XGBoost

In [8]:
import xgboost as xgb
import time

In [9]:
start = time.time()

# 데이터를 XGBoost 모델에 맞게 변환
xgb_dtrain = xgb.DMatrix(data = train_x, label = train_y)
xgb_dtest = xgb.DMatrix(data = test_x)

xgb_param = {'max_depth': 10, # 트리 깊이
             'learning_rate': 0.01, # Step Size
             'n_estimators': 100, # Number of trees, 트리 생성 개수
             'objective': 'multi:softmax', # 목적 함수
             'num_class': len(set(train_y)) + 1} # 파라미터 추가, Label must be in [0, num_class] -> num_class보다 1 커야한다.

xgb_model = xgb.train(params = xgb_param, dtrain = xgb_dtrain)
xgb_model_predict = xgb_model.predict(xgb_dtest)

print("Accuracy: %.2f" % (accuracy_score(test_y, xgb_model_predict) * 100), "%")
print("Time: %.2f" % (time.time() - start), "seconds")

[12:58:26] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Accuracy: 76.67 %
Time: 8.28 seconds


In [10]:
xgb_model_predict

array([5., 3., 6., ..., 9., 2., 7.], dtype=float32)

## 2. LightGBM

In [11]:
import lightgbm as lgb

start = time.time()
lgb_dtrain = lgb.Dataset(data = train_x, label = train_y) # 학습 데이터를 LightGBM 모델에 맞게 변환

lgb_param = {'max_depth': 10,
            'learning_rate': 0.01,
            'n_estimators': 100,         # Number of trees, 트리 생성 개수
            'objective': 'multiclass',   # 목적 함수
            'num_class': len(set(train_y)) + 1} # num_class보다 1 커야한다.

lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain)
lgb_model_predict = np.argmax(lgb_model.predict(test_x), axis = 1)

print("Accuracy: %.2f" % (accuracy_score(test_y, lgb_model_predict) * 100), "%")
print("Time: %.2f" % (time.time() - start), "seconds")

C:\Anaconda3\envs\py37_env\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Accuracy: 73.57 %
Time: 6.04 seconds


In [12]:
lgb_model.predict(test_x)

array([[9.14195396e-16, 2.27101123e-02, 3.85432853e-01, ...,
        3.25763637e-02, 7.61804989e-02, 4.64683926e-02],
       [1.16264426e-15, 3.77262272e-02, 2.26739830e-01, ...,
        1.92812290e-01, 1.01826669e-01, 8.11988908e-02],
       [8.03061518e-16, 1.74144816e-02, 1.18616633e-01, ...,
        2.57008295e-02, 6.69196402e-02, 4.08194769e-02],
       ...,
       [8.35633463e-16, 5.03899675e-02, 1.67414994e-01, ...,
        4.81925245e-02, 1.02188157e-01, 3.95359380e-01],
       [1.01560525e-15, 2.13472780e-02, 4.60509516e-01, ...,
        3.32864538e-02, 8.46310485e-02, 5.16230380e-02],
       [8.92775603e-16, 2.04756016e-02, 1.31867900e-01, ...,
        4.13697353e-01, 1.44381292e-01, 4.53796283e-02]])

## 3. Catboost

In [13]:
import catboost as cb

start = time.time() # 시작 시간 지정

cb_dtrain = cb.Pool(data = train_x, label = train_y) # 학습 데이터를 Catboost 모델에 맞게 변환

cb_param = {'max_depth': 10, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 100, # Number of trees, 트리 생성 개수
            'verbose': False,
            'eval_metric': 'Accuracy', # 평가 척도
            'loss_function': 'MultiClass'} # 손실 함수, 목적 함수

cb_model = cb.train(pool = cb_dtrain, params = cb_param)
cb_model_predict = np.argmax(cb_model.predict(test_x), axis = 1) + 1

print("Accuracy: %.2f" % (accuracy_score(test_y, cb_model_predict) * 100), "%")
print("Time: %.2f" % (time.time() - start), "seconds")

Accuracy: 69.64 %
Time: 57.39 seconds


In [14]:
cb_model.predict(test_x)

array([[-0.35426047,  1.22109587,  0.44230101, ..., -0.1698448 ,
        -0.02059177, -0.2130643 ],
       [-0.07235138,  0.42535181,  0.20060428, ...,  0.21863604,
         0.2719157 ,  0.25089315],
       [-0.3315885 , -0.31862353, -0.31279765, ..., -0.29798357,
        -0.24018767, -0.32984969],
       ...,
       [ 0.05304325,  0.02500267, -0.14752573, ..., -0.20741963,
         0.12789417,  1.51166757],
       [-0.55093666,  1.7691278 ,  0.99746884, ..., -0.3420542 ,
        -0.49799871, -0.38136323],
       [-0.3033724 ,  0.09352675, -0.11808658, ...,  0.65825036,
         1.05515787, -0.20799899]])

# Predict house price

In [15]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

In [16]:
# 데이터 불러오기
data = pd.read_csv("./data/kc_house_data.csv") 
data.head() # 데이터 확인

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [17]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1) # 불필요한 변수 제거

In [18]:
feature_columns = list(data.columns.difference(['price'])) # Price를 제외한 모든 행
X = data[feature_columns]
y = data['price']

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(15129, 8) (6484, 8) (15129,) (6484,)


In [19]:
start = time.time() # 시작 시간 지정

lgb_dtrain = lgb.Dataset(data = train_x, label = train_y) # 학습 데이터를 LightGBM 모델에 맞게 변환
lgb_param = {'max_depth': 10, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 500, # Number of trees, 트리 생성 개수
            'objective': 'regression'} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.

lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain)

C:\Anaconda3\envs\py37_env\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [20]:
sqrt(mean_squared_error(lgb_model.predict(test_x),test_y))

210904.17249451784

## Ensemble의 Ensemble

In [21]:
import random

bagging_predict_result = [] # 빈 리스트 생성
for _ in range(10):
    data_index = [data_index for data_index in range(train_x.shape[0])] # 학습 데이터의 인덱스를 리스트로 변환
    random_data_index = np.random.choice(data_index, train_x.shape[0]) # 데이터의 1/10 크기만큼 랜덤 샘플링, // 는 소수점을 무시하기 위함
    print(len(set(random_data_index)))
    
    lgb_dtrain = lgb.Dataset(data = train_x.iloc[random_data_index,], label = train_y.iloc[random_data_index,])
    lgb_param = {'max_depth': 14, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 500, # Number of trees, 트리 생성 개수
            'objective': 'regression'} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
    
    lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행
    predict1 = lgb_model.predict(test_x) # 테스트 데이터 예측
    bagging_predict_result.append(predict1) # 반복문이 실행되기 전 빈 리스트에 결과 값 저장

9637


C:\Anaconda3\envs\py37_env\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


9574
9492
9578
9497
9637
9596
9555
9492
9591


In [22]:
bagging_predict_result

[array([541654.5023765 , 611994.92621911, 890167.31493284, ...,
        352022.17256582, 907514.0477077 , 442314.40183782]),
 array([ 513087.97137473,  600356.84132344, 1007488.51486898, ...,
         326107.56587351,  848018.06987567,  466615.87320833]),
 array([503268.03213063, 625202.07962185, 935232.66490756, ...,
        329207.11808109, 929200.59398651, 477364.16696889]),
 array([495642.53040411, 623035.71739593, 966333.79825412, ...,
        338237.48297809, 927400.40105155, 476370.56673325]),
 array([479784.42486111, 666180.78892091, 912602.12442397, ...,
        343553.30674019, 923559.56255802, 463624.68446121]),
 array([491577.57466963, 639252.68876003, 939456.06913863, ...,
        330478.87959973, 879543.17998065, 462549.95543173]),
 array([472229.92215722, 600225.03759248, 967611.17694941, ...,
        340423.04734016, 924920.80942808, 467808.10617296]),
 array([489531.48966869, 649719.42640237, 893651.23663977, ...,
        345273.6411187 , 935465.56883684, 468028.304140

In [23]:
# Bagging을 바탕으로 예측한 결과값에 대한 평균을 계산
bagging_predict = [] # 빈 리스트 생성

for lst2_index in range(test_x.shape[0]): # 테스트 데이터 개수만큼의 반복
    temp_predict = [] # 임시 빈 리스트 생성 (반복문 내 결과값 저장)
    for lst_index in range(len(bagging_predict_result)): # Bagging 결과 리스트 반복
        temp_predict.append(bagging_predict_result[lst_index][lst2_index]) # 각 Bagging 결과 예측한 값 중 같은 인덱스를 리스트에 저장
    bagging_predict.append(np.mean(temp_predict)) # 해당 인덱스의 30개의 결과값에 대한 평균을 최종 리스트에 추가

In [24]:
# 예측한 결과값들의 평균을 계산하여 실제 테스트 데이트의 타겟변수와 비교하여 성능 평가

print("RMSE: {}".format(sqrt(mean_squared_error(bagging_predict, test_y)))) # RMSE

RMSE: 210203.8862839774


In [27]:
bagging_predict[:10]

[502063.7107472263,
 627110.5295839779,
 945924.0655033311,
 1568757.2240713646,
 636904.2457191136,
 367903.5861467939,
 689993.2320537636,
 430039.65328048926,
 462766.9140390955,
 496078.51605035755]